In [ ]:
import math
import pandas as pd
import numpy as np
import time
import csv
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.static import players






def getdataset(dataset, step):
    X, Y = [], []
    for i in range(len(dataset) - step - 1):
        a = dataset[i:(i+step), 0]
        X.append(a)
        Y.append(dataset[i+step,0])
    return np.array(X), np.array(Y)

def model(data):

    if len(data) % 2 != 0:
        data = data[1:]

    sequence_length = int(len(data)*.5)  # Length of input sequences

    #data = dataset[:sequence_length]
    test_sequence = np.array(data[sequence_length:])


    X = []
    y = []

    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    # Reshape the input data to match LSTM input shape (samples, time steps, features)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build the LSTM model
    model = Sequential([
        LSTM(units=50, activation='relu', input_shape=(sequence_length, 1)),
        Dense(units=1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=100, batch_size=1)

    # Predict the next number based on a sequence
    test_sequence = np.reshape(test_sequence, (1, sequence_length, 1))
    predicted_next_number = model.predict(test_sequence)
    return predicted_next_number

def predict(dataset):
    predictions = []
    for i in range(0,5):
        prediction = model(dataset)
        predictions.append(prediction)
        
    next_prediction = np.mean(predictions)
    return next_prediction


def valora(stat):
    stat = stat[~np.isnan(stat)]
    prediction = predict(stat)
    return prediction


def plot(data, prediction, y_label, title):


    x_data = np.arange(1, len(data) + 1)
    x = 0
    for i in x_data:
        if x == 0:
            plt.plot(i, data[x], 'ko', label = "Historical " + title)

        else:
            plt.plot(i, data[x], 'ko')
        x = x+1

    plt.plot(len(data)+1, prediction, 'mo', label = "Prediction for " + title)
    plt.xlabel("Games")
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()
    plt.show()




def main():


    dataset_test = [1,5,2,7,3,12,6,14,8,20,14,32,15,25,15]
    #prediction = predict(dataset_test, "test")
    #print(prediction)

    #gamelog_bron_all = playergamelog.PlayerGameLog(player_id='2544', season=SeasonAll.all)
    #gamelog_bron = playergamelog.PlayerGameLog(player_id='2544', season='2023')
    #df_bron_games_2024 = gamelog_bron.get_data_frames()
    #df_bron_all = gamelog_bron_all.get_data_frames()
    #bron_points_all = np.array(df_bron_all[0]['PTS'])




    #players we want to look into
   # superstars = ["Stephen Curry", "Michael Jordan", "Kevin Durant",
          #       "Lebron James", "Allen Iverson"]

    #initialize empty lists that will put player id into
    #player_ids = []
    #for i in superstars:
    #   player_ids.append(players.find_players_by_full_name(i)[0]['id'])



    print("****Welcome to Valora****")
    print("Enter 0 to exit")

    while True:
        player_name = (input("Enter Full Name of Player to Predict: "))
        if player_name == 0:
            break

        player_id = players.find_players_by_full_name(player_name)[0]['id']
        print("Player data found for: ", player_name, " Player Id: ", player_id)

        prediction_statistic = input("Enter statstic to predict FGA	FTA	'FGM', 'FGA', 'FTM', 'FTA', 'FG3M', 'FG3A' AST STL BLK TOV PF PTS: ")
        if prediction_statistic == 0:
            break


        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2024')
        gamelog_data = gamelog.get_data_frames()

        out = csv.writer(open("/Users/home/Desktop/output.csv", "w"), delimiter=',', quoting=csv.QUOTE_ALL)
        out.writerow(gamelog_data)
        out.writerow(gamelog_data)



        player_stat = np.array(gamelog_data[0][prediction_statistic])

        print("Data received for Player")


        print("Predicting for 2024-2025 Season...")
        time.sleep(3)
        prediction = valora(player_stat)
        print("Prediction for ", player_name, " ", prediction_statistic, " ", prediction)
        plot(player_stat, prediction, prediction_statistic, player_name + " " + prediction_statistic )



main()


****Welcome to Valora****
Enter 0 to exit


Enter Full Name of Player to Predict:  jalen brunson


Player data found for:  jalen brunson  Player Id:  1628973


Enter statstic to predict FGA	FTA	'FGM', 'FGA', 'FTM', 'FTA', 'FG3M', 'FG3A' AST STL BLK TOV PF PTS:  PTS


Data received for Player
Predicting for 2024-2025 Season...
Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 464.7452  
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 116.4252
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 91.5999
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 130.1577
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 130.3844
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 200.8216 
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 69.6114
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 75.6672
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63.4515
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 80.4139
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 73.0960
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 65.3169
Epoch 13/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 76.5635
Epoch 14/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 87.1477
Epoch 15/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1